Belief propagation

In [3]:
import numpy as np

class GPSNode:
    def __init__(self, lat, lon, uncertainty):
        self.lat = lat
        self.lon = lon
        self.uncertainty = uncertainty

    def message(self):
        return (self.lat, self.lon, self.uncertainty)

class BeliefPropagationGPS:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def compute_confidence(self):
        weighted_lat, weighted_lon, total_weight = 0, 0, 0

        for node in self.nodes:
            lat, lon, uncertainty = node.message()
            weight = 1 / uncertainty
            weighted_lat += lat * weight
            weighted_lon += lon * weight
            total_weight += weight

        estimated_lon = weighted_lon / total_weight
        estimated_lat = weighted_lat / total_weight
        estimated_lon = weighted_lon / total_weight
        confidence = total_weight / len(self.nodes)

        return estimated_lat, estimated_lon, confidence

# Example usage
bp_gps = BeliefPropagationGPS()
bp_gps.add_node(GPSNode(40.7128, -74.0060, 0.1))  # Example node with (lat, lon, uncertainty)
bp_gps.add_node(GPSNode(40.7130, -74.0070, 0.2))  # Another example node

estimated_lat, estimated_lon, confidence = bp_gps.compute_confidence()
print(f"Estimated Location: Latitude {estimated_lat}, Longitude {estimated_lon}, Confidence {confidence}")


Estimated Location: Latitude 40.71286666666666, Longitude -74.00633333333333, Confidence 7.5


In [2]:
import numpy as np

class GPSNode:
    def __init__(self, lat, lon, variance):
        self.lat = lat
        self.lon = lon
        self.variance = variance  # Assume variance is same for lat and lon for simplicity

    def message(self):
        return (self.lat, self.lon, self.variance)

class BeliefPropagationGPS:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def compute_confidence(self):
        n = len(self.nodes)
        if n == 0:
            return None

        measurement_matrix = np.zeros((n * 2, 1))  # 2 coordinates per node
        covariance_matrix = np.zeros((n * 2, n * 2))

        for i, node in enumerate(self.nodes):
            lat, lon, variance = node.message()
            measurement_matrix[i*2:i*2+2, 0] = [lat, lon]
            # Assuming the covariance between lat and lon is zero for simplicity
            covariance_matrix[i*2:i*2+2, i*2:i*2+2] = np.diag([variance, variance])

        # Add covariance between different GPS measurements
        # This is where domain-specific knowledge is required to set these values correctly
        # For this example, we'll assume a fixed covariance value for all pairs
        fixed_covariance = 0.01  # Example value
        for i in range(n):
            for j in range(i+1, n):
                covariance_matrix[i*2, j*2] = fixed_covariance
                covariance_matrix[j*2, i*2] = fixed_covariance
                covariance_matrix[i*2+1, j*2+1] = fixed_covariance
                covariance_matrix[j*2+1, i*2+1] = fixed_covariance

        # Compute the combined estimate
        covariance_inv = np.linalg.inv(covariance_matrix)
        weighted_avg = covariance_inv.dot(measurement_matrix)
        estimated_location = np.sum(weighted_avg, axis=1) / np.sum(covariance_inv)

        # Compute the confidence metric
        confidence = 1 / np.sqrt(np.linalg.det(covariance_matrix))

        return estimated_location[0], estimated_location[1], confidence

    def report_to_network(self):
        lat, lon, confidence = self.compute_confidence()
        report = {
            "estimated_latitude": lat,
            "estimated_longitude": lon,
            "confidence_metric": confidence
        }
        # Code to send this report to the network
        # network_api.send(report)
        print("Report to network:", report)

# Example usage
bp_gps = BeliefPropagationGPS()
bp_gps.add_node(GPSNode(40.7128, -74.0060, 0.01))  # Example node
bp_gps.add_node(GPSNode(40.7130, -74.0070, 0.02))  # Another example node

bp_gps.report_to_network()


Report to network: {'estimated_latitude': 20.3563, 'estimated_longitude': -37.0025, 'confidence_metric': 9999.99999999999}


mariam's sandbox

In [13]:
# first, we simulate GPS data that can be used for belief propagation
# TODO: implement longitude latitude type node measurements - for now, we will
# assume Gaussian distributed measurements from GPS

import random
import time

random.seed(0)

def GPS_node_stream(mu, sigma):
    # TODO: noisy data creation
    while True:
        yield random.normalvariate(mu, sigma)  # Simulate sensor readings

def process_data(data):
    std = np.std(data)
    mean = np.mean(data)
    print("Processing:", data)

sensor = GPS_node_stream(5, 0.1)
window = []

for _ in range(100):  # Run for more iterations to see the sliding window in action
    data_point = next(sensor)
    
    if len(window) < 10:
        window.append(data_point)
    else:
        process_data(window)
        window = window[1:]  # Remove the oldest data point
        window.append(data_point)
    
    time.sleep(1)  # Simulate a delay, e.g., 1 second between data points

# Don't forget to process the last set of data
if window:
    process_data(window)

Processing: [4.981613177890674, 5.003250412907758, 5.069882769325398, 4.990363215498473, 5.141350779004812, 4.846736412890912, 5.027093664976886, 4.879446561569581, 4.9946858657362565, 4.970977572937749]
Processing: [5.003250412907758, 5.069882769325398, 4.990363215498473, 5.141350779004812, 4.846736412890912, 5.027093664976886, 4.879446561569581, 4.9946858657362565, 4.970977572937749, 4.970717688313624]
Processing: [5.069882769325398, 4.990363215498473, 5.141350779004812, 4.846736412890912, 5.027093664976886, 4.879446561569581, 4.9946858657362565, 4.970977572937749, 4.970717688313624, 4.789261161560858]
Processing: [4.990363215498473, 5.141350779004812, 4.846736412890912, 5.027093664976886, 4.879446561569581, 4.9946858657362565, 4.970977572937749, 4.970717688313624, 4.789261161560858, 5.008473482917045]
Processing: [5.141350779004812, 4.846736412890912, 5.027093664976886, 4.879446561569581, 4.9946858657362565, 4.970977572937749, 4.970717688313624, 4.789261161560858, 5.008473482917045,

In [ ]:
# next we want to find the covariance matrix between different input signals from different GPS nodes
# TODO: generalise stream to have as many data points as we need from different inputs: first 2, then further i.e. find 10 x 10 covariance for 10 nodes

def covariance_mat(window1, window2):
    return np.cov(window1, window2)

# create graph

class GPSNode:
    def __init__(self, measurement, uncertainty): # assuming a very simple input format for now- TODO: modify to add nuance in different reporting measures/mechanisms
        self.measurement = measurement
        self.uncertainty = uncertainty

    def message(self):
        return (self.measurement, self.uncertainty)
    
class BeliefPropagation:
    def __init__(self):
        self.nodes = []

    def add_node(self, node):
        self.nodes.append(node)

    def run_belief_propagation(self):
        n = len(self.nodes)
        if n == 0:
            return None
        
        # covariance between nodes that are connected to the same factor